In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!apt-get update
!apt-get install -y git

In [ ]:
!git clone https://github.com/meronheron/Chain-of-Tools.git

In [ ]:
!ls /kaggle/working

In [ ]:
!ls /kaggle/working/Chain-of-Tools

In [ ]:
!sed -i '/torch==2.2.1+cu118/d; /torchaudio==2.2.1+cu118/d; /torchvision==0.17.1+cu118/d' /kaggle/working/Chain-of-Tools/requirements.txt

In [ ]:
!pip install -r /kaggle/working/Chain-of-Tools/requirements.txt

In [ ]:
!pip install torchvision==0.17.1

In [ ]:
!pip install bitsandbytes

In [ ]:
!pip list | grep -E 'accelerate|beautifulsoup4|einops|huggingface-hub|numpy|safetensors|tokenizers|torchvision'

In [ ]:
!ls /kaggle/working/Chain-of-Tools/src

In [ ]:
!ls /kaggle/working/Chain-of-Tools/config

In [ ]:
!mkdir -p /kaggle/working/config
!cp /kaggle/working/Chain-of-Tools/config/basic.yaml /kaggle/working/config/
!mkdir -p /kaggle/working/my_log

In [ ]:
!sed -i 's|checkpoint: ../PTM/Llama-2-7b-chat-hf|checkpoint: mistralai/Mistral-7B-Instruct-v0.2|' /kaggle/working/config/basic.yaml

In [ ]:
!pip install huggingface_hub


In [ ]:
from kaggle_secrets import UserSecretsClient; 
from huggingface_hub import login; 

login(token=UserSecretsClient().get_secret('HF_TOKEN'))

In [ ]:
!cat /kaggle/working/config/basic.yaml | grep checkpoint

In [ ]:
!ls /kaggle/working
!ls /kaggle/working/Chain-of-Tools/src/script

In [ ]:
!ls Chain-of-Tools/src/script/

In [ ]:
!find /kaggle/working/Chain-of-Tools -name "WillMindS"

In [ ]:
!ls /kaggle/working/Chain-of-Tools/src/WillMindS

In [ ]:
!sed -i 's|checkpoint: ../PTM/Llama-2-7b-chat-hf|checkpoint: mistralai/Mistral-7B-Instruct-v0.2|' /kaggle/working/Chain-of-Tools/config/train_judge.yaml
!sed -i 's/half_quantized: True/half_quantized: False/' /kaggle/working/Chain-of-Tools/config/train_judge.yaml
!sed -i 's/train_batch: 4/train_batch: 2/' /kaggle/working/Chain-of-Tools/config/train_judge.yaml
!sed -i 's/train_batch: 8/train_batch: 2/' /kaggle/working/Chain-of-Tools/config/train_judge.yaml
!sed -i 's/gradient_accumulation_steps: 16/gradient_accumulation_steps: 32/' /kaggle/working/Chain-of-Tools/config/train_judge.yaml
!sed -i 's|meta-llama/Llama-2-7b-chat-hf|mistralai/Mistral-7B-Instruct-v0.2|' /kaggle/working/Chain-of-Tools/config/train_judge.yaml
!sed -i 's/model_name: LLaMA/model_name: Mistral/' /kaggle/working/Chain-of-Tools/config/train_judge.yaml

In [ ]:
!cat /kaggle/working/Chain-of-Tools/config/train_judge.yaml

In [ ]:

!pip install transformers==4.41.2

In [ ]:
!sed -i 's|from transformers import MistralTokenizer; self.tokenizer = MistralTokenizer.from_pretrained(self.config.checkpoint, use_fast=True)|from transformers import AutoTokenizer; self.tokenizer = AutoTokenizer.from_pretrained(self.config.checkpoint, use_fast=True, trust_remote_code=True)|' /kaggle/working/Chain-of-Tools/src/model.py
!sed -i 's|from transformers import LlamaTokenizer; self.tokenizer = LlamaTokenizer.from_pretrained(self.config.checkpoint, use_fast=True)|from transformers import AutoTokenizer; self.tokenizer = AutoTokenizer.from_pretrained(self.config.checkpoint, use_fast=True, trust_remote_code=True)|' /kaggle/working/Chain-of-Tools/src/model.py
!sed -i 's|self.tokenizer(tool_data, return_tensors="pt", add_special_tokens=False)|self.tokenizer(tool_data, return_tensors="pt")|' /kaggle/working/Chain-of-Tools/src/model.py

In [ ]:
!sed -i 's|self.foundation_model = AutoModelForCausalLM.from_pretrained(self.config.checkpoint, device_map="auto")|from transformers import BitsAndBytesConfig, AutoModelForCausalLM; import torch; quantization_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16); self.foundation_model = AutoModelForCausalLM.from_pretrained(self.config.checkpoint, device_map="auto", quantization_config=quantization_config); self.foundation_model.gradient_checkpointing_enable()|' /kaggle/working/Chain-of-Tools/src/model.py

In [ ]:
!sed -i 's|.to("cuda:1")|.to("cuda:0")|' /kaggle/working/Chain-of-Tools/src/model.py

In [ ]:
!sed -i 's|self.retriever_tool_selection = MLPLayer(self.hidden_size, self.intermediate_size, self.retrieval_size).to("cuda:0")|self.retriever_tool_selection = MLPLayer(self.hidden_size, self.intermediate_size, self.retrieval_size).to("cuda:0").to(torch.float16)|' /kaggle/working/Chain-of-Tools/src/model.py


In [ ]:
!sed -i s|self.tool_judge = MLPLayer(self.hidden_size, self.intermediate_size, 3).to("cuda:0")|self.tool_judge = MLPLayer(self.hidden_size, self.intermediate_size, 3).to("cuda:0").to(torch.float16)|' /kaggle/working/Chain-of-Tools/src/model.py

In [ ]:
!sed -i 's|probs = self.tool_judge(hidden_states.to(next(self.tool_judge.parameters()).device))|self.tool_judge.to(dtype=torch.float16)\n        device = next(self.tool_judge.parameters()).device\n        probs = self.tool_judge(hidden_states.to(device=device, dtype=torch.float16))|' /kaggle/working/Chain-of-Tools/src/model.py

In [ ]:
!sed -i '/def forward(self, x):/{n;s|.*down_proj = self.down_proj.*|        x = x.to(dtype=torch.float16)\n        down_proj = self.down_proj(self.act_fn(self.gate_proj(x)) * self.up_proj(x))|}' /kaggle/working/Chain-of-Tools/src/model.py

In [ ]:
!cat /kaggle/working/Chain-of-Tools/src/model.py | grep -A 5 "def forward(self, x):"

In [ ]:
!cat /kaggle/working/Chain-of-Tools/src/train.py | grep -A 14 "loss = F.binary_cross_entropy"

In [ ]:
# Suppress XLA warnings
import os
os.environ["XLA_FLAGS"] = "--xla_gpu_enable_cudnn_autotune=false"

In [ ]:
!cat /kaggle/working/Chain-of-Tools/config/train_judge.yaml

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
!ls -R /kaggle/working/Chain-of-Tools/data/gsm8k_xl

In [ ]:
!mkdir -p /kaggle/working/Chain-of-Tools/my_log

In [ ]:
!cd /kaggle/working/Chain-of-Tools && PYTHONPATH=/kaggle/working/Chain-of-Tools/src PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True python src/script/train_gsm8k_judge.py --config_file config/train_judge.yaml

In [ ]:

#!cd /kaggle/working/Chain-of-Tools && PYTHONPATH=/kaggle/working/Chain-of-Tools/src python src/script/test_gsm8k.py --config_file config/test.yaml